In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from ingest.catalog import DatasetCatalog
from ingest.loader import RawDatasetLoader
from ingest.fetch.csv import CsvAdapter
from ingest.fetch.sct import SocrataAdapter
from ingest.fetch.excel import ExcelAdapter
from ingest.fetch.metadata import SocrataMetadata

In [3]:
## servicios
catalog = DatasetCatalog()
loader = RawDatasetLoader(
    csv_adapter=CsvAdapter(),
    sct_adapter=SocrataAdapter(),
    excel_adapter=ExcelAdapter(),
)

In [4]:
ds = catalog.get("divipola")
meta = loader.metadata(ds)
if isinstance(meta, SocrataMetadata):
    print("Nombre:", meta.name)
    print("Descripción:", meta.description)
    print("Columnas:")
    columns = meta.columns
    for col in columns:
        print(f"  - {col.name} ({col.fieldName}): {col.dataTypeName}")
else:
    print("Este origen no soporta metadata estructurada.")

Nombre: DIVIPOLA- Códigos municipios
Descripción: Código de la división Político Administrativa del país, Actualización a corte 30 diciembre 2024.
Columnas:
  - Código Departamento (cod_dpto): text
  - Nombre Departamento (dpto): text
  - Código Municipio (cod_mpio): text
  - Nombre Municipio (nom_mpio): text
  - Tipo: Municipio / Isla / Área no municipalizada (tipo_municipio): text
  - longitud (longitud): text
  - Latitud (latitud): text


In [5]:
records = list(loader.load(ds))
df = pd.DataFrame(records)
df.head(3)

,cod_dpto,dpto,cod_mpio,nom_mpio,tipo_municipio,longitud,latitud
0,05,ANTIOQUIA,05001,MEDELLÍN,Municipio,"-75,581775","6,246631"
1,05,ANTIOQUIA,05002,ABEJORRAL,Municipio,"-75,428739","5,789315"
2,05,ANTIOQUIA,05004,ABRIAQUÍ,Municipio,"-76,064304","6,632282"


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122 entries, 0 to 1121
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cod_dpto        1122 non-null   object
 1   dpto            1122 non-null   object
 2   cod_mpio        1122 non-null   object
 3   nom_mpio        1122 non-null   object
 4   tipo_municipio  1122 non-null   object
 5   longitud        1122 non-null   object
 6   latitud         1122 non-null   object
dtypes: object(7)
memory usage: 61.5+ KB


In [7]:
cols = [
    "longitud",
    "latitud"
]

pretty_names = {
    col.fieldName: col.name
    for col in meta.columns
}

def pretty(col):
    return pretty_names.get(col, col)

# Convertir a número
for c in cols:
    df[c] = df[c].astype(str)\
                 .str.replace("$", "", regex=False)\
                 .str.replace(".", "", regex=False)\
                 .str.replace(",", ".", regex=False)\
                 .astype(float) 
    df[c] = pd.to_numeric(df[c], errors='coerce')


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122 entries, 0 to 1121
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   cod_dpto        1122 non-null   object 
 1   dpto            1122 non-null   object 
 2   cod_mpio        1122 non-null   object 
 3   nom_mpio        1122 non-null   object 
 4   tipo_municipio  1122 non-null   object 
 5   longitud        1122 non-null   float64
 6   latitud         1122 non-null   float64
dtypes: float64(2), object(5)
memory usage: 61.5+ KB


In [9]:
df.head()

,cod_dpto,dpto,cod_mpio,nom_mpio,tipo_municipio,longitud,latitud
0,05,ANTIOQUIA,05001,MEDELLÍN,Municipio,-75.581775,6.246631
1,05,ANTIOQUIA,05002,ABEJORRAL,Municipio,-75.428739,5.789315
2,05,ANTIOQUIA,05004,ABRIAQUÍ,Municipio,-76.064304,6.632282
3,05,ANTIOQUIA,05021,ALEJANDRÍA,Municipio,-75.141346,6.376061
4,05,ANTIOQUIA,05030,AMAGÁ,Municipio,-75.702188,6.038708
